# Gjentakelser i et utvalg norsk lyrikk



## Innledning 

I et stort utvalg dikt fra 1890-tallet finner vi at et overveldende flertall, hele 83%, er strofiske. Av disse igjen er halvparten på 4 linjer. 

Ettersom diktenes strofer er annotert med enderimsmønstre hver for seg, og ikke alle strofene i ett og samme dikt har fått samme rimmønster, kan vi se på fordelingen av enderimsmønstre over alle strofer, og finner at hele 1400 forskjellige rimmønstre fordeler seg utover strofene i litt over 1700 dikt. Mange av disse mønstrene forekommer bare én gang og kan dermed egentlig ikke kalles rimmønstre i tradisjonell forstand, men springer ut av annotasjonene som er blitt gjort ved hjelp av automatisk fonemisk transkripsjon og en regelbasert algoritme som annoterer verselinjer med en ny bokstav for hver linje som ikke rimer med noen foregående verselinjer i samme strofe. 

De vanligste mønstrene blant strofer med enderim er abcb (1169 av alle strofene, i 451 eller ca. 25% av alle diktene), abab (757 strofer, der 315 av diktene har minst en strofe med dette mønsteret), og abac (i 432 strofer, og i 234 av diktene), mens aabb kun forekommer i 312 strofer (ca X %).

Om vi lar mønsteret som forekommer hyppigst i hvert dikt få bli diktets enderimsmønster, finner vi at fordelingen endrer seg litt: 

I 208 dikt har flest strofer enderim som følger abcb, mens det nest vanligste er abab (188 dikt). I 74 dikt er aabb det vanligste mønsteret, og abac gjelder for 62 dikt. 

I 751 av diktene forekommer det mest frekvente enderimsmønsteret i bare en av strofene → vi trenger mer informasjon om hvor mange strofer disse diktene har, og hva slags andre type scheme som finnes i hvert av diktene og om de er strofiske eller ikke (for å etterprøve metoden og transkripsjonen), MEN det kan være et spor for å finne alle de irregulære frie versene. 




## Materiale

(1/2 side)

NORN Dikt: metadata og oversikt


## Metode 

(maks 2 sider)
beskrivelse av 

1. enderimsannotering 
2. alliterasjon 
3. anafor (ref. Rhetorical Figure Detection)



## Analyse
(6 sider, maks 8 inkl. visualisering)
- enkeltdikt + regelbaserte algoritmer
- visualiseringer: 
    - strofiske dikt: bilde av diktene oppå hverandre, jf. CHR 2024


## Konklusjon
(1/2 side)

# KODE 


## Enderim

In [1]:
import poetry_analysis as pa

import pandas as pd 
from pathlib import Path


# Definer datamappen
DATADIR =  Path('/home/ingeridd/prosjekter/dikt-utforskning/data')

DATADIR.exists()


False

## Annoter enderim for tekst og transkripsjoner

Det første vi gjør er å lese inn diktene og annotere enderimene, og laste resultatene inn i en dataramme

In [4]:
# Lister med diktfilene
poem_txt_files = list(DATADIR.glob('**/*.txt'))
poem_json_files = list(DATADIR.glob('**/*østnorsk.json'))


In [7]:
from poetry_analysis import rhyme_detection as rd

def load_rhyme_df(filepath: Path) -> pd.DataFrame:
    filename = filepath.stem
    pronfile = filepath.parent / f"{filename}_østnorsk.json"

    if not pronfile.exists():
        raise FileNotFoundError(f"Pronunciation file {pronfile} does not exist.")
    
    df_pron = pd.DataFrame(rd.tag_poem_file(pronfile)).explode("verses").reset_index(drop=True)

    df_text = pd.DataFrame(rd.tag_poem_file(filepath)).explode("verses").reset_index(drop=True)

    text_verses = df_text.verses.apply(pd.Series)
    transcribed_verses = df_pron.verses.apply(pd.Series)

    verses = transcribed_verses.drop(columns=["text", "tokens"]).merge(text_verses[["text", "tokens", "last_token", "rhyme_tag", "rhymes_with", "rhyme_score"]], left_index=True, right_index=True, suffixes=("_syll", "_text"))

    df = df_pron[["stanza_id", "rhyme_scheme"]].merge(verses, left_index=True, right_index=True)
    df = df.rename(columns={"stanza_id": "stanza", "rhyme_scheme": "rhyme", "verse_id": "verse"})

    df["filename"] = filename
    df["poem"] = filename.split("_")[0]
    ordered_columns = [
        "filename", "poem",
        'stanza', 'rhyme', 'verse', 
        'rhymes_with_text', 'rhymes_with_syll', 
        'rhyme_score_text', 'rhyme_score_syll', 
        'rhyme_tag_text', 'rhyme_tag_syll', 
        'last_token_text',  'last_token_syll', 
        'text', 'transcription', 
        'tokens', 'syllables',
    ]
    return df[ordered_columns]



In [8]:
dfs = [load_rhyme_df(textfile) for textfile in poem_txt_files]

IndexError: list index out of range

In [ ]:
from collections import Counter

# Hent ut de mest brukte rimmønstrene for alle strofene 
scheme_counter = Counter([pattern for schemes in rhymepatterns.values() for pattern in schemes])

scheme_counter.most_common(20)

In [ ]:
# Se nærmere på antall dikt som har et bestemt rimmønster
most_common_schemes = [poem_id for poem_id, schemes in rhymepatterns.items() if "abcd" in schemes]
len(most_common_schemes)

In [ ]:
# Hent ut rimmønsteret som forekommer hyppigst i hvert dikt, og frekvensen til det rimmønsteret
majority_schemes =  [{"poem_id":poem_id, "scheme": Counter(schemes).most_common(1)} for poem_id, schemes in rhymepatterns.items()]

df = pd.DataFrame(majority_schemes)

df["pattern"] = df.scheme.explode()
df[["scheme", "count"]] = df["pattern"].apply(pd.Series)

df.drop(columns=["pattern"], inplace=True)
df

In [ ]:
## Tell antall dikt de ulike mønstrene er vanligst i
df.scheme.value_counts()

In [ ]:
# Hent ut dikt som ikke har enderim, med mønsteret "abcd" (hver linje har en ny bokstav, dvs. ingen linjer rimer med noen foregående i samme strofe)
# Filtrer vekk diktene der dette mønsteret ikke forekommer mer enn én gang
df[(df.scheme == "abcd") & (df["count"] > 1)].sort_values(by="count", ascending=False)

In [ ]:
## TODO: Legg til kolonne med info om antall strofer of hvilke andre schemes som er brukt i diktet

# Hent ut diktene der majoritetsrimmønsteret kun forekommer 1 gang, dvs. at det ikke er noe klart rimmønster
df[df["count"] == 1]

In [ ]:
# Hent frekvensen til alle rimmønstrene i hvert dikt
counted = [{"poem_id":poem_id, "scheme": Counter(schemes)} for poem_id, schemes in rhymepatterns.items()]

df = pd.DataFrame(counted)

pd.merge(df["poem_id"], df["scheme"].apply(pd.Series), left_index=True, right_index=True)

In [ ]:
df.scheme.str.len().value_counts()

In [ ]:
df.scheme.str.len().sort_values(ascending=False)

In [ ]:
df[df.scheme.str.len() == 330]

## Hvilke ord rimer oftest? 

Vi har annotasjonene fra rimtaggeren liggende som json-filer, og leser dem inn på nytt for å hente ut siste ord i hver linje for hvert dikt. 

In [ ]:
from poetry_analysis import rhyme_detection as rd 
from poetry_analysis import utils

logging.basicConfig(level=logging.DEBUG)




def get_stanzas_from_transcription(transcription: dict) -> list:
    """Parse a dict of transcribed verse lines and return a list of stanzas."""
    n_lines = len(transcription.keys()) - 2  # subtract the text_id and dialect keys
    logging.debug("Number of lines in poem: %s", n_lines)
    poem = []
    stanza = []
    for n in range(n_lines):
        verse = transcription.get(f"line_{n}")
        if (verse is not None) and (len(verse) > 0):
            syllables = utils.syllabify(verse)
            stanza.append(syllables)
        else:
            if len(stanza) == 0:
                continue
            poem.append(stanza)
            stanza = []
    if len(poem) == 0 and len(stanza) > 0:
        poem.append(stanza)
    return poem


def annotate_rhyming_patterns(poem: dict) -> list:
    rhyming_patterns = []
    stanzas = get_stanzas_from_transcription(poem)    
    
    for stanza in enumerate(stanzas):
        tagged = rd.tag_rhyming_verses(stanza)
        rhyme_scheme = rd.collate_rhyme_scheme(tagged)
        rhyming_patterns.append(rhyme_scheme)
    return rhyming_patterns



def fetch_end_words(poem: dict) -> list:
    """Hent ordene på slutten av hver linje i hver strofe i diktet. Hver strofe er en liste av ord."""
    n_lines = len(poem) - 2 # subtract the text_id and dialect keys
    end_words = []
    stanza_words = []
    for idx in range(n_lines):
        line = poem.get(f"line_{idx}")
        if (line is not None) and (len(line) > 0):
            words = [word[0] for word in line]
            stanza_words.append(words[-1])
        else:
            if not stanza_words: 
                continue
            end_words.append(stanza_words)
            stanza_words = []
    return end_words

# Hent ut rimordene fra diktene 
def fetch_rhyming_words(folder: Path) -> dict:
    rhyming_words = {}    
    for filename in folder.glob("**/*_østnorsk.json"):
        poem_id = filename.stem.split("_")[0]
        poem = json.loads(filename.read_text())
        
        poem_title = poem.get("text_id")
        try: 
            patterns = rhymepatterns[poem_id]
        except KeyError:
            try: 
                patterns = annotate_rhyming_patterns(poem)
            except TypeError:
                logging.error("No rhyme patterns found for poem %s", poem_title)
                logging.error(poem)
                continue
        end_words = fetch_end_words(poem)
        rhyming_words[poem_id] = {"title": poem_title, "patterns": patterns, "end_words": end_words}
    return rhyming_words


rhyming_words = fetch_rhyming_words(folder)

In [ ]:
len(rhyming_words)

In [ ]:
word_pairs = Counter()

for poem_id, poem in rhyming_words.items():
    patterns = poem["patterns"]
    end_words = poem["end_words"]

    for pattern, word in zip(patterns, end_words):
         # Dette er 1 strofe
        #if pattern not in ["abac", "abab", "aabb", "abcb"]:
        #    continue
        rhyme_pairs = {}
        for letter, w in zip(pattern, word):
            # Dette er 1 linje 
           # print(poem_id, letter, w)
            if letter in rhyme_pairs:
                rhyme_pairs[letter].append(w)
            else:
                rhyme_pairs[letter] = [w]
        #print(rhyme_pairs)
        rhyme_pairs = [tuple(v) for _, v in rhyme_pairs.items() if len(v) > 1]
    
        word_pairs.update(rhyme_pairs)

word_pairs.most_common(20)

In [ ]:
# Antall rimordgrupper
len(word_pairs)

In [ ]:
# Se på rimordgrupper som har flere enn 2 ord 
with Path("output/rimordgrupper.csv").open("w") as fp: 
    fp.write("word_group\n")
    for key in word_pairs.keys():
        if len(key) > 2:
            fp.write(f"{','.join(key)}\n")

#[key for key in word_pairs.keys() if len(key) > 2]


In [ ]:
with Path("output/word_pairs.csv").open("w") as fp: 
    for key, value in word_pairs.most_common():
        fp.write(f"{value},{','.join(key)}\n")